In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import requests 
import pandas as pd 
from web3 import Web3
from pyvis.network import Network
import pyvis.network as net


In [19]:
def number_to_readable(num):
    abs_num = abs(num)
    if abs_num < 1000:
        return str(num)
    elif abs_num < 1000000:
        return f"{num/1000:.1f}k".replace('.0', '')
    elif abs_num < 1000000000:
        return f"{num/1000000:.1f}M".replace('.0', '')
    elif abs_num < 1000000000000:
        return f"{num/1000000000:.1f}B".replace('.0', '')
    else:
        return "∞"

# Markets

In [2]:

market_query = """
query{
  markets(first: 1000) {
    items {
      uniqueKey
      loanAsset {
        symbol
        decimals
        priceUsd
      }
      state{
        borrowAssetsUsd
        supplyAssetsUsd
      }
      collateralAsset{
        symbol
      }
      lltv
      morphoBlue{chain{network}}
    }
  }
}
"""

In [3]:
blue_api_url = "https://blue-api.morpho.org/graphql"

params = {
    "query": market_query
}

response = requests.post(blue_api_url, json=params)
market_raw_data = response.json()['data']['markets']['items']


In [4]:
market_data = []

for market in market_raw_data:
    loan_asset = market['loanAsset']['symbol']
    if market['collateralAsset'] is not None:
        collateral_asset = market['collateralAsset']['symbol']
    else:
        collateral_asset = "Idle"
    lltv = int(market['lltv']) / 1e16
    chain = market['morphoBlue']['chain']['network']
    loan_decimals = market['loanAsset']['decimals']
    loan_price_usd = market['loanAsset']['priceUsd']
    supplyAssetsUsd = market['state']['supplyAssetsUsd']
    market_name = f"{collateral_asset}/{loan_asset} ({lltv}%)"
    market_id = market['uniqueKey']
    market_data.append([market_name,  chain, market_id, loan_decimals, loan_price_usd, supplyAssetsUsd])

market_data = pd.DataFrame(market_data, columns=['market_name', 'market_chain',  'market_id' , 'loan_decimals', 'loan_price_usd', 'supply_assets_usd'])

# MetaMorpho Vaults

In [5]:
vault_query = """
query{
  vaults(first: 1000) {
    items {
      address
      name
      id
      chain {
        network
      }
      state {
        totalAssetsUsd
      }
    }
  }
}
"""

In [6]:

params = {
    "query": vault_query
}

response = requests.post(blue_api_url, json=params)
response_code = response.status_code
vault_raw_data = response.json()['data']['vaults']['items']


In [7]:

vault_data = []

for vault in vault_raw_data:
    vault_name = vault['name']
    chain = vault['chain']['network']
    address = vault['address']
    vault_id = vault['id']
    total_assets_usd = vault['state']['totalAssetsUsd']
    vault_data.append([vault_name, chain, address, total_assets_usd])

vault_data = pd.DataFrame(vault_data, columns=['vault_name', 'vault_chain', 'vault_address', 'total_assets_usd'])


# Flows

In [8]:

# os.environ['RPC_URL'] = '<YOUR_RPC_URL>'
w3 = Web3(Web3.HTTPProvider(os.environ['RPC_URL']))


In [9]:
public_allocator_address = "0xfd32fA2ca22c76dD6E550706Ad913FC6CE91c75D"

with open('abis/public_allocator_abi.json') as f:
    public_allocator_abi = json.load(f)

public_allocator = w3.eth.contract(address=public_allocator_address, abi=public_allocator_abi)





In [10]:

set_flow_caps_events = public_allocator.events.SetFlowCaps.create_filter(fromBlock=0).get_all_entries()

flows = []
for event in set_flow_caps_events:
    vault = event['args']['vault']
    block_number = event['blockNumber']
    for flow_cap in event['args']['config']:
        market_id, max_in, max_out = '0x' + flow_cap['id'].hex(), flow_cap['caps']['maxIn'], flow_cap['caps']['maxOut']
        flows.append((vault, block_number, market_id, max_in, max_out))

flow_df = pd.DataFrame(flows, columns=['vault_address', 'block_number', 'market_id', 'max_in', 'max_out'])
flow_df['market_id'] = flow_df['market_id'].apply(lambda x: x.lower()).astype(str)

In [11]:
flow_df = pd.merge(flow_df, market_data, on='market_id', how='left')

In [12]:
flow_df = pd.merge(flow_df, vault_data, on='vault_address', how='left')
flow_df['max_in'] = flow_df['max_in'].astype(float) / 10**flow_df['loan_decimals']
flow_df['max_out'] = flow_df['max_out'].astype(float) / 10**flow_df['loan_decimals']
flow_df['max_in_usd'] = flow_df['max_in'] * flow_df['loan_price_usd']
flow_df['max_out_usd'] = flow_df['max_out'] * flow_df['loan_price_usd']


# Create a directed graph

In [24]:

# Create a pyvis Network object
net = Network(notebook=True, directed=True)

scale = 1e6
# Add nodes for unique market_ids and vault_ids
market_ids = flow_df['market_name'].unique()
vault_ids = flow_df['vault_name'].unique()

for market_id in market_ids:
    value = market_data[market_data['market_name'] == market_id]['supply_assets_usd'].values[0]
    net.add_node(str(market_id), label=f"Market {market_id}", color='#ff8000', size=value/scale, level=0)

for vault_id in vault_ids:
    value = vault_data[vault_data['vault_name'] == vault_id]['total_assets_usd'].values[0]
    net.add_node(str(vault_id), label=f"Vault {vault_id}", color='#0000FF', size=value/scale, level=1)

# Add edges
MAX_VALUE = 1e8

for _, row in flow_df.iterrows():
    if row['max_in_usd'] > 0:
        
        net.add_edge(str(row['market_name']), str(row['vault_name']), 
                    title=f"Max In: ${number_to_readable(row['max_in_usd'])}",
                    value=min(row['max_in_usd'], MAX_VALUE), color="#ff0000")
    if row['max_out_usd'] > 0:
        net.add_edge(str(row['vault_name']), str(row['market_name']), 
                    title=f"Max Out: ${number_to_readable(row['max_out_usd'])}",
                    value=min(row['max_out_usd'], MAX_VALUE), color="#00ff00")
        

# Save and display the graph
net.show('blueflow.html')

blueflow.html
